# Test Network Hyperparameters

In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


In [40]:
df = pd.read_csv('finalDF.csv',index_col=0)
df = df.drop(columns=['glucose'])
display(df.head(5))

,hadm_id,subject_id,expire_flag,admission_type,elix_score,age,admission_type_encoded,glucose_readings,0,1,...,6,7,8,9,10,11,12,13,14,range
0,137275,9319,0,ELECTIVE,12,82,0,45,118.0,97.0,...,93.0,93.0,109.0,109.0,94.0,94.0,98.0,98.0,85.0,82.0
3,101422,71582,0,EMERGENCY,0,76,1,21,106.0,134.0,...,115.0,114.0,159.0,161.0,90.0,113.0,113.0,148.0,139.0,88.0
5,148291,9299,0,EMERGENCY,-2,62,1,28,222.0,128.0,...,120.0,106.0,106.0,97.0,137.0,126.0,107.0,111.0,111.0,145.0
6,116534,46467,0,ELECTIVE,13,82,0,37,235.0,235.0,...,78.0,84.0,99.0,76.0,114.0,152.0,202.0,262.0,304.0,228.0
8,133497,20374,0,ELECTIVE,15,56,0,28,89.0,89.0,...,74.0,74.0,87.0,93.0,83.0,0.0,97.0,94.0,94.0,122.0


In [41]:
from sklearn.utils import resample
df_majority = df[df.expire_flag==0]
df_minority = df[df.expire_flag==1]

df_minority_upsampled = resample(df_minority, 
                                 replace=True,   
                                 n_samples=7436,     
                                 random_state=123) 
df_upsampled = pd.concat([df_minority_upsampled, df_majority])

In [42]:
pydf = df_upsampled.values
print(pydf.shape)
X = pydf[:,4:24]
y = df_upsampled['expire_flag']
#scaler = MinMaxScaler(feature_range=(0, 1))
#X = scaler.fit_transform(X)
print(X)

(14872, 24)
[[12 85 1 ... 169.0 169.0 105.0]
 [19 56 1 ... 71.0 99.0 194.0]
 [7 72 1 ... 105.0 129.0 219.0]
 ...
 [12 49 1 ... 75.0 76.0 171.0]
 [25 65 1 ... 295.0 303.0 283.0]
 [10 63 1 ... 297.0 413.0 340.0]]


In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=1)

In [46]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

def mortality_model():
    model = Sequential()
    model.add(Dense(14, input_dim=20, activation='relu'))
    model.add(Dense(7,activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
    
model = KerasClassifier(build_fn=mortality_model,epochs=50,batch_size=32)   

In [47]:
model.fit(X_train,y_train)

Epoch 1/50
10744/10744 [==============================] - 1s 63us/step - loss: 1.0956 - acc: 0.4990
Epoch 2/50
10744/10744 [==============================] - 0s 23us/step - loss: 0.6941 - acc: 0.4990
Epoch 3/50
10744/10744 [==============================] - 0s 23us/step - loss: 0.6920 - acc: 0.4996
Epoch 4/50
10744/10744 [==============================] - 0s 23us/step - loss: 0.6921 - acc: 0.5012
Epoch 5/50
10744/10744 [==============================] - 0s 23us/step - loss: 0.6915 - acc: 0.5019
Epoch 6/50
10744/10744 [==============================] - 0s 23us/step - loss: 0.6912 - acc: 0.5032
Epoch 7/50
10744/10744 [==============================] - 0s 23us/step - loss: 0.6911 - acc: 0.5080
Epoch 8/50
10744/10744 [==============================] - 0s 23us/step - loss: 0.6893 - acc: 0.5165
Epoch 9/50
10744/10744 [==============================] - 0s 24us/step - loss: 0.6887 - acc: 0.5186
Epoch 10/50
10744/10744 [==============================] - 0s 23us/step - loss: 0.6859 - acc: 0.5199

batch size and epochs

In [52]:
from sklearn.model_selection import GridSearchCV

model = KerasClassifier(build_fn=mortality_model)   
batch_size = [10,20,50,100]
epochs = [5,10,50]
parameters = {'batch_size': batch_size,'epochs': epochs}
clf = GridSearchCV(model,parameters)
clf.fit(X_train,y_train)

Epoch 1/5
7162/7162 [==============================] - 1s 133us/step - loss: 1.3918 - acc: 0.5505
Epoch 2/5
7162/7162 [==============================] - 0s 69us/step - loss: 0.7186 - acc: 0.5553
Epoch 3/5
7162/7162 [==============================] - 0s 69us/step - loss: 0.7000 - acc: 0.5602
Epoch 4/5
7162/7162 [==============================] - 0s 68us/step - loss: 0.6963 - acc: 0.5635
Epoch 5/5
7162/7162 [==============================] - 0s 35us/step
Epoch 1/5
7163/7163 [==============================] - 1s 135us/step - loss: 0.8827 - acc: 0.4775
Epoch 2/5
7163/7163 [==============================] - 1s 73us/step - loss: 0.6913 - acc: 0.5329
Epoch 3/5
7163/7163 [==============================] - 1s 71us/step - loss: 0.6820 - acc: 0.5840
Epoch 4/5
7163/7163 [==============================] - 0s 70us/step - loss: 0.6758 - acc: 0.5912
Epoch 5/5
7163/7163 [==============================] - 0s 38us/step
Epoch 1/5
7163/7163 [==============================] - 1s 143us/step - loss: 1.1000 - 

7162/7162 [==============================] - 1s 75us/step - loss: 0.6479 - acc: 0.6350
Epoch 32/50
7162/7162 [==============================] - 1s 77us/step - loss: 0.6517 - acc: 0.6231
Epoch 33/50
7162/7162 [==============================] - 1s 75us/step - loss: 0.6470 - acc: 0.6289
Epoch 34/50
7162/7162 [==============================] - 1s 76us/step - loss: 0.6392 - acc: 0.6396
Epoch 35/50
7162/7162 [==============================] - 1s 75us/step - loss: 0.6370 - acc: 0.6481
Epoch 36/50
7162/7162 [==============================] - 1s 75us/step - loss: 0.6369 - acc: 0.6456
Epoch 37/50
7162/7162 [==============================] - 1s 75us/step - loss: 0.6342 - acc: 0.6438
Epoch 38/50
7162/7162 [==============================] - 1s 75us/step - loss: 0.6344 - acc: 0.6361
Epoch 39/50
7162/7162 [==============================] - 1s 75us/step - loss: 0.6309 - acc: 0.6518
Epoch 40/50
7162/7162 [==============================] - 1s 75us/step - loss: 0.6259 - acc: 0.6574
Epoch 41/50
7162/7162 

7163/7163 [==============================] - 1s 77us/step - loss: 0.6210 - acc: 0.6642
Epoch 12/50
7163/7163 [==============================] - 1s 77us/step - loss: 0.6148 - acc: 0.6697
Epoch 13/50
7163/7163 [==============================] - 1s 77us/step - loss: 0.6168 - acc: 0.6704
Epoch 14/50
7163/7163 [==============================] - 1s 77us/step - loss: 0.6076 - acc: 0.6722
Epoch 15/50
7163/7163 [==============================] - 1s 77us/step - loss: 0.6059 - acc: 0.6767
Epoch 16/50
7163/7163 [==============================] - 1s 77us/step - loss: 0.6014 - acc: 0.6818
Epoch 17/50
7163/7163 [==============================] - 1s 76us/step - loss: 0.6031 - acc: 0.6850
Epoch 18/50
7163/7163 [==============================] - 1s 77us/step - loss: 0.5970 - acc: 0.6813
Epoch 19/50
7163/7163 [==============================] - 1s 77us/step - loss: 0.5952 - acc: 0.6855
Epoch 20/50
7163/7163 [==============================] - 1s 77us/step - loss: 0.5953 - acc: 0.6926
Epoch 21/50
7163/7163 

7163/7163 [==============================] - 1s 176us/step - loss: 2.4967 - acc: 0.5136
Epoch 2/10
7163/7163 [==============================] - 0s 46us/step - loss: 0.7652 - acc: 0.5181
Epoch 3/10
7163/7163 [==============================] - 0s 48us/step - loss: 0.7233 - acc: 0.5252
Epoch 4/10
7163/7163 [==============================] - 0s 45us/step - loss: 0.7104 - acc: 0.5241
Epoch 5/10
7163/7163 [==============================] - 0s 47us/step - loss: 0.7030 - acc: 0.5425
Epoch 6/10
7163/7163 [==============================] - 0s 45us/step - loss: 0.6919 - acc: 0.5616
Epoch 7/10
7163/7163 [==============================] - 0s 46us/step - loss: 0.6833 - acc: 0.5742
Epoch 8/10
7163/7163 [==============================] - 0s 44us/step - loss: 0.6790 - acc: 0.5865
Epoch 9/10
7163/7163 [==============================] - 0s 44us/step - loss: 0.6772 - acc: 0.5912
Epoch 10/10
7163/7163 [==============================] - 0s 29us/step
Epoch 1/50
7162/7162 [==============================] - 1s

7163/7163 [==============================] - 0s 46us/step - loss: 8.0872 - acc: 0.4983
Epoch 22/50
7163/7163 [==============================] - 0s 46us/step - loss: 8.0872 - acc: 0.4983
Epoch 23/50
7163/7163 [==============================] - 0s 44us/step - loss: 8.0872 - acc: 0.4983
Epoch 24/50
7163/7163 [==============================] - 0s 45us/step - loss: 8.0872 - acc: 0.4983
Epoch 25/50
7163/7163 [==============================] - 0s 44us/step - loss: 8.0872 - acc: 0.4983
Epoch 26/50
7163/7163 [==============================] - 0s 45us/step - loss: 8.0872 - acc: 0.4983
Epoch 27/50
7163/7163 [==============================] - 0s 54us/step - loss: 8.0872 - acc: 0.4983
Epoch 28/50
7163/7163 [==============================] - 0s 54us/step - loss: 8.0872 - acc: 0.4983
Epoch 29/50
7163/7163 [==============================] - 0s 55us/step - loss: 8.0872 - acc: 0.4983
Epoch 30/50
7163/7163 [==============================] - 0s 53us/step - loss: 8.0872 - acc: 0.4983
Epoch 31/50
7163/7163 

...

7163/7163 [==============================] - 0s 22us/step - loss: 0.7056 - acc: 0.5586
Epoch 12/50
7163/7163 [==============================] - 0s 24us/step - loss: 0.7202 - acc: 0.5383
Epoch 13/50
7163/7163 [==============================] - 0s 24us/step - loss: 0.7146 - acc: 0.5362
Epoch 14/50
7163/7163 [==============================] - 0s 22us/step - loss: 0.7021 - acc: 0.5496
Epoch 15/50
7163/7163 [==============================] - 0s 22us/step - loss: 0.6981 - acc: 0.5545
Epoch 16/50
7163/7163 [==============================] - 0s 23us/step - loss: 0.7027 - acc: 0.5498
Epoch 17/50
7163/7163 [==============================] - 0s 22us/step - loss: 0.7013 - acc: 0.5494
Epoch 18/50
7163/7163 [==============================] - 0s 22us/step - loss: 0.6915 - acc: 0.5718
Epoch 19/50
7163/7163 [==============================] - 0s 22us/step - loss: 0.6995 - acc: 0.5555
Epoch 20/50
7163/7163 [==============================] - 0s 22us/step - loss: 0.6908 - acc: 0.5745
Epoch 21/50
7163/7163 

7163/7163 [==============================] - 0s 14us/step - loss: 8.0287 - acc: 0.5019
Epoch 3/10
7163/7163 [==============================] - 0s 13us/step - loss: 8.0287 - acc: 0.5019
Epoch 4/10
7163/7163 [==============================] - 0s 14us/step - loss: 8.0287 - acc: 0.5019
Epoch 5/10
7163/7163 [==============================] - 0s 14us/step - loss: 8.0287 - acc: 0.5019
Epoch 6/10
7163/7163 [==============================] - 0s 14us/step - loss: 8.0287 - acc: 0.5019
Epoch 7/10
7163/7163 [==============================] - 0s 13us/step - loss: 8.0287 - acc: 0.5019
Epoch 8/10
7163/7163 [==============================] - 0s 14us/step - loss: 8.0287 - acc: 0.5019
Epoch 9/10
7163/7163 [==============================] - 0s 14us/step - loss: 8.0287 - acc: 0.5019
Epoch 10/10
7163/7163 [==============================] - 0s 9us/step
Epoch 1/50
7162/7162 [==============================] - 2s 223us/step - loss: 4.3728 - acc: 0.5561
Epoch 2/50
7162/7162 [==============================] - 0s 

7163/7163 [==============================] - 0s 14us/step - loss: 0.6652 - acc: 0.6197
Epoch 23/50
7163/7163 [==============================] - 0s 13us/step - loss: 0.6629 - acc: 0.6207
Epoch 24/50
7163/7163 [==============================] - 0s 13us/step - loss: 0.6620 - acc: 0.6263
Epoch 25/50
7163/7163 [==============================] - 0s 14us/step - loss: 0.6601 - acc: 0.6298
Epoch 26/50
7163/7163 [==============================] - 0s 13us/step - loss: 0.6589 - acc: 0.6282
Epoch 27/50
7163/7163 [==============================] - 0s 13us/step - loss: 0.6561 - acc: 0.6333
Epoch 28/50
7163/7163 [==============================] - 0s 13us/step - loss: 0.6551 - acc: 0.6335
Epoch 29/50
7163/7163 [==============================] - 0s 13us/step - loss: 0.6538 - acc: 0.6365
Epoch 30/50
7163/7163 [==============================] - 0s 13us/step - loss: 0.6500 - acc: 0.6451
Epoch 31/50
7163/7163 [==============================] - 0s 13us/step - loss: 0.6480 - acc: 0.6465
Epoch 32/50
7163/7163 

10744/10744 [==============================] - 1s 133us/step - loss: 0.6860 - acc: 0.5455
Epoch 3/50
10744/10744 [==============================] - 1s 123us/step - loss: 0.6702 - acc: 0.5940
Epoch 4/50
10744/10744 [==============================] - 1s 129us/step - loss: 0.6576 - acc: 0.6158
Epoch 5/50
10744/10744 [==============================] - 1s 127us/step - loss: 0.6448 - acc: 0.6372
Epoch 6/50
10744/10744 [==============================] - 1s 122us/step - loss: 0.6286 - acc: 0.6579
Epoch 7/50
10744/10744 [==============================] - 1s 124us/step - loss: 0.6182 - acc: 0.6683 1s - l
Epoch 8/50
10744/10744 [==============================] - 1s 123us/step - loss: 0.6125 - acc: 0.6742
Epoch 9/50
10744/10744 [==============================] - 1s 122us/step - loss: 0.6072 - acc: 0.6787
Epoch 10/50
10744/10744 [==============================] - 1s 118us/step - loss: 0.6066 - acc: 0.6749
Epoch 11/50
10744/10744 [==============================] - ETA: 0s - loss: 0.5997 - acc: 0.686

GridSearchCV(cv=None, error_score='raise',
       estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x000001ECF4EEEBE0>,
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'batch_size': [10, 20, 50, 100], 'epochs': [5, 10, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [55]:
print(clf.best_score_,clf.best_params_)

#means = clf.cv_results_['mean_test_score']
#paramters = clf.cv_results_['params']
#for mean, parameter in zip(means,parameters):
#    print(mean, parameter)


0.7052308276159743 {'batch_size': 10, 'epochs': 50}


In [57]:
def mortality_model(optimizer):
    model = Sequential()
    model.add(Dense(14, input_dim=20, activation='relu'))
    model.add(Dense(7,activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=mortality_model,epochs=50,batch_size=10)  
parameters = {'optimizer':['SGD','RMSprop','Adagrad','Adam','Adamax','Nadam']}
clf = GridSearchCV(model,parameters)
clf.fit(X_train,y_train)

Epoch 1/50
7162/7162 [==============================] - 2s 337us/step - loss: 7.9399 - acc: 0.5018
Epoch 2/50
7162/7162 [==============================] - 1s 116us/step - loss: 7.9423 - acc: 0.5018
Epoch 3/50
7162/7162 [==============================] - 1s 117us/step - loss: 7.9423 - acc: 0.5018
Epoch 4/50
7162/7162 [==============================] - 1s 111us/step - loss: 7.9423 - acc: 0.5018
Epoch 5/50
7162/7162 [==============================] - 1s 114us/step - loss: 7.9423 - acc: 0.5018
Epoch 6/50
7162/7162 [==============================] - 1s 113us/step - loss: 7.9423 - acc: 0.5018
Epoch 7/50
7162/7162 [==============================] - 1s 112us/step - loss: 7.9423 - acc: 0.5018
Epoch 8/50
7162/7162 [==============================] - 1s 113us/step - loss: 7.9423 - acc: 0.5018
Epoch 9/50
7162/7162 [==============================] - 1s 114us/step - loss: 7.9423 - acc: 0.5018
Epoch 10/50
7162/7162 [==============================] - 1s 118us/step - loss: 7.9423 - acc: 0.5018
Epoch 11/

Epoch 32/50
7163/7163 [==============================] - 1s 112us/step - loss: 8.0872 - acc: 0.4983
Epoch 33/50
7163/7163 [==============================] - 1s 113us/step - loss: 8.0872 - acc: 0.4983
Epoch 34/50
7163/7163 [==============================] - 1s 112us/step - loss: 8.0872 - acc: 0.4983
Epoch 35/50
7163/7163 [==============================] - 1s 111us/step - loss: 8.0872 - acc: 0.4983
Epoch 36/50
7163/7163 [==============================] - 1s 112us/step - loss: 8.0872 - acc: 0.4983
Epoch 37/50
7163/7163 [==============================] - 1s 110us/step - loss: 8.0872 - acc: 0.4983
Epoch 38/50
7163/7163 [==============================] - 1s 113us/step - loss: 8.0872 - acc: 0.4983
Epoch 39/50
7163/7163 [==============================] - 1s 122us/step - loss: 8.0872 - acc: 0.4983
Epoch 40/50
7163/7163 [==============================] - 1s 122us/step - loss: 8.0872 - acc: 0.4983
Epoch 41/50
7163/7163 [==============================] - 1s 115us/step - loss: 8.0872 - acc: 0.4983


KeyboardInterrupt: 